In [1]:
%config Completer.use_jedi=False
import torch
import numpy as np
import apex.amp as amp
import torchvision as tv
import matplotlib.pyplot as plt

import pytorch_tools as pt
from pytorch_tools.utils.misc import count_parameters

from src.dali_dataloader import DaliLoader

# Evaluate

In [2]:
INP = torch.ones(2, 3, 224, 224).cuda()
PRED = torch.ones(2, 1000).cuda()
LOSS = pt.losses.CrossEntropyLoss().cuda()
METRIC = [pt.metrics.Accuracy(), pt.metrics.Accuracy(5)]
@torch.no_grad()
def evaluate_model(model, size=224):
    """Eval model on ImageNet validation dataset"""
    BS = 25 if size >= 384 else 125
    val_loader = DaliLoader(False, bs=BS, sz=size, workers=12)
    model = model.eval().cuda()
    model = amp.initialize(model, opt_level='O1', verbosity=0)
    runner = pt.fit_wrapper.Runner(model, None, LOSS, metrics=METRIC, callbacks=None)
    _, (acc1, acc5) = runner.evaluate(val_loader)
    print(f"Acc@1: {acc1:.2f}. Acc@5: {acc5:.2f}")
    return acc1, acc5

In [3]:
m1 = pt.models.resnet50(pretrained="imagenet").cuda()

In [4]:
evaluate_model(m1)

Acc@1: 75.99. Acc@5: 92.96


(75.9900013256073, 92.96000143051147)